### Importaciones

In [1]:
import os
from dotenv import load_dotenv
from openai import AsyncAzureOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner
from agents.model_settings import ModelSettings
from IPython.display import display, Markdown
from agents import function_tool
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField
from azure.search.documents.indexes import SearchIndexClient

In [2]:
load_dotenv(override=True)

True

### Variables de entorno

In [3]:
azure_openai_key = os.getenv('AZURE_OPENAI_KEY')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')

azure_openai_gpt_deployment = os.getenv('AZURE_OPENAI_GPT_DEPLOYMENT')
azure_openai_gpt_version = os.getenv('AZURE_OPENAI_GPT_VERSION')
azure_openai_gpt_model = os.getenv('AZURE_OPENAI_GPT_MODEL')

azure_openai_embedding_deployment = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT')
azure_openai_embedding_version = os.getenv('AZURE_OPENAI_EMBEDDING_VERSION')
azure_openai_embedding_model = os.getenv('AZURE_OPENAI_EMBEDDING_MODEL')

azure_search_key = os.getenv('AZURE_SEARCH_KEY')
azure_search_endpoint = os.getenv('AZURE_SEARCH_ENDPOINT')
azure_search_index = os.getenv('AZURE_SEARCH_INDEX')

### Creación de index por única vez

In [4]:
index = SearchIndex(
    name=azure_search_index,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        SearchField(name="page_embedding_text_ada", type="Collection(Edm.Single)", stored=False, vector_search_dimensions=1536, vector_search_profile_name="hnsw_text_ada"),
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw_text_ada", algorithm_configuration_name="alg", vectorizer_name="azure_openai_text_ada")],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_ada",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,
                    deployment_name=azure_openai_embedding_deployment,
                    model_name=azure_openai_embedding_model
                )
            )
        ]
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[
                        SemanticField(field_name="page_chunk")
                    ]
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint=azure_search_endpoint, credential=AzureKeyCredential(azure_search_key))
index_client.create_or_update_index(index)
print(f"Index '{azure_search_index}' created or updated successfully")

Index 'farmaceutica' created or updated successfully


### Clientes

In [4]:
client = AsyncAzureOpenAI(
    api_key=azure_openai_key,
    api_version=azure_openai_gpt_version,
    azure_endpoint=azure_openai_endpoint
)

search_client = SearchClient(
    endpoint=azure_search_endpoint, 
    index_name=azure_search_index, 
    credential=AzureKeyCredential(azure_search_key)
)

### Tools

In [5]:
@function_tool
def search_documents_tool(query: str):
    """Busca en el índice y devuelve los chunks más relevantes."""
    try:
        results = search_client.search(
            search_text=query,
            select=["page_chunk", "page_number"],
            query_type="semantic",
            query_caption="extractive",
            top=3
        )
        relevant_chunks = [f"Página {result['page_number']}: {result['page_chunk']}" for result in results]
        return "\n---\n".join(relevant_chunks)
    except Exception as e:
        return f"Error al buscar: {e}"

### Agente conocimiento

In [6]:
knowledge_agent = Agent(
    name="Agente buscador",
    instructions="""
    Eres un agente especializado en la recuperación de información. Tu única función es tomar la pregunta del usuario,
    ejecutar la herramienta 'search_documents' con esa pregunta y devolver el texto relevante que encontraste.
    """,
    model=OpenAIChatCompletionsModel(
        model=azure_openai_gpt_deployment,
        openai_client=client,
    ),
    tools=[search_documents_tool],
    model_settings=ModelSettings(tool_choice="required")
)

### Agente farmacéutico

In [7]:
pharma_agent = Agent(
    name="Agente farmacéutico",
    instructions="""
    Eres un experto en documentación farmacéutica. Recibes un contexto de búsqueda.
    Utiliza **exclusivamente** el contexto proporcionado para responder a la pregunta del usuario de manera concisa y profesional.
    Si el contexto no contiene la respuesta, di que no puedes responder basándote en la información disponible.
    """,
    model=OpenAIChatCompletionsModel(
        model=azure_openai_gpt_deployment,
        openai_client=client,
    ),
    model_settings=ModelSettings(temperature=0.1)
)

### Ejecución del agente

In [8]:
async def main():
    # pregunta = "¿En qué módulo está el CPP?"
    # pregunta = "¿En qué módulo puedo encontrar estudios en humanos?"
    pregunta = "¿Cuál es el nombre del producto mencionado en el PoA?"
    
    # 1. El knowledge_agent busca la información.
    contexto_recuperado = await Runner.run(knowledge_agent, input=pregunta)
  
    # 2. El pharma_agent recibe la pregunta original y el contexto
    result = await Runner.run(pharma_agent, input=pregunta, context=contexto_recuperado.final_output)

    display(Markdown(result.final_output))

await main()

OPENAI_API_KEY is not set, skipping trace export


No puedo responder basándome en la información disponible.

OPENAI_API_KEY is not set, skipping trace export
